In [1]:
import requests
import json
import os
import shutil
import pwd
import grp

# Deploying on namespace

## Creating the namespace
Creating the folder for the namespace

In [2]:
default_services_path="/usr/lib/cgi-bin"
my_namespace_path="/opt/zooservices_namespaces/eoepca"

In [3]:
if not os.path.exists(my_namespace_path):
    os.makedirs(my_namespace_path) 

Copying the Deploy and Undeploy services in the namespace folder

In [4]:
shutil.copyfile(f"{default_services_path}/DeployProcess.py", f"{my_namespace_path}/DeployProcess.py")
shutil.copyfile(f"{default_services_path}/DeployProcess.zcfg", f"{my_namespace_path}/DeployProcess.zcfg")
shutil.copyfile(f"{default_services_path}/UndeployProcess.py", f"{my_namespace_path}/UndeployProcess.py")
shutil.copyfile(f"{default_services_path}/UndeployProcess.zcfg", f"{my_namespace_path}/UndeployProcess.zcfg")

'/opt/zooservices_namespaces/eoepca/UndeployProcess.zcfg'

Granting permissions to the namespace folder

In [5]:
user_id=pwd.getpwnam("www-data")[2]
group_id=grp.getgrnam('www-data')[2]

In [6]:
for dirpath, dirnames, filenames in os.walk(my_namespace_path):
    shutil.chown(dirpath, user_id, group_id)
    for filename in filenames:
        shutil.chown(os.path.join(dirpath, filename), user_id,group_id)

## Inspect namespace services
Provides a list of all services in the namespace *eoepca*

In [7]:
response = requests.get('http://localhost/eoepca/ogc-api/processes')
response.json()

{'processes': [{'id': 'UndeployProcess',
   'title': 'Undeploy Process',
   'description': 'This method will undeploy a deployed processing service.',
   'version': '1.0.0',
   'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
   'outputTransmission': ['value', 'reference'],
   'links': [{'rel': 'self',
     'type': 'application/json',
     'title': 'Process Description',
     'href': 'http://localhost/eoepca/ogc-api/processes/UndeployProcess'},
    {'rel': 'alternate',
     'type': 'text/html',
     'title': 'Process Description',
     'href': 'http://localhost/eoepca/ogc-api/processes/UndeployProcess.html'}]},
  {'id': 'DeployProcess',
   'title': 'Deploys a zoo service from an application package.',
   'description': 'Deploys a zoo service from an application package.',
   'version': '2.0.0',
   'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
   'outputTransmission': ['value', 'reference'],
   'links': [{'rel': 'self',
     'type': 'application/js

## Deploying application package

In [8]:
url = "http://localhost/eoepca/ogc-api/processes/"
payload = open('data/dnbr_app/dnbr.cwl', 'rb').read()

headers = {'Content-Type': 'application/cwl'}
response = requests.request("POST", url, headers=headers, data=payload)

print(f"Body: {response.json()}\n")
print(f"Headers: {response.headers}")
print(f"Status code: {response.status_code}")

Body: {'message': 'Service dnbr version 0.1.0 successfully deployed.', 'service': 'dnbr', 'status': 'success'}

Headers: {'Date': 'Wed, 03 Aug 2022 09:31:39 GMT', 'Server': 'Apache/2.4.41 (Ubuntu)', 'Content-Length': '104', 'Location': 'http://localhost/eoepca/ogc-api/processes/dnbr', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'application/json; charset=UTF-8'}
Status code: 201


In [9]:
response = requests.get('http://localhost/eoepca/ogc-api/processes/dnbr')
response.json()

{'id': 'dnbr',
 'title': 'dNBR - produce the delta normalized difference between NIR and SWIR 22 over a pair of stac items',
 'description': 'dNBR - produce the delta normalized difference between NIR and SWIR 22 over a pair of stac items',
 'version': '0.1.0',
 'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
 'outputTransmission': ['value', 'reference'],
 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
   'type': 'application/json',
   'title': 'Execute End Point',
   'href': 'http://localhost/eoepca/ogc-api/processes/dnbr/execution'},
  {'rel': 'alternate',
   'type': 'text/html',
   'title': 'Execute End Point',
   'href': 'http://localhost/eoepca/ogc-api/processes/dnbr/execution.html'}],
 'inputs': {'pre_stac_item': {'title': 'pre_stac_item',
   'description': 'Pre-event Sentinel-2 item',
   'schema': {'type': 'string', 'default': 'Any value'}},
  'post_stac_item': {'title': 'post_stac_item',
   'description': 'Post-event Sentinel-2 item',
   

## Execute the application

In [138]:
url = "http://localhost/eoepca/ogc-api/processes/dnbr"

payload = open('data/dnbr_app/dnbr_execute_payload.json', 'rb').read()
headers = {
  'Accept': 'application/json',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(f"Headers: {response.headers}\n")
print(f"Status code: {response.status_code}")

Headers: {'Date': 'Wed, 03 Aug 2022 15:10:09 GMT', 'Server': 'Apache/2.4.41 (Ubuntu)', 'Location': 'http://localhost/eoepca/ogc-api/jobs/5d49062c-133e-11ed-b6ba-0242ac190102', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json;charset=UTF-8'}

Status code: 201


In [139]:
location = response.headers["Location"]
print(location)

http://localhost/eoepca/ogc-api/jobs/5d49062c-133e-11ed-b6ba-0242ac190102


## Get Job Status

In [140]:
get_status_response = requests.get(location)
get_status_response.json()

{'jobID': '5d49062c-133e-11ed-b6ba-0242ac190102',
 'status': 'failed',
 'message': 'Unable to run the Service. The message returned back by the Service was the following: No message provided',
 'links': [{'title': 'Status location',
   'rel': 'status',
   'type': 'application/json',
   'href': 'http://localhost/eoepca/ogc-api/jobs/5d49062c-133e-11ed-b6ba-0242ac190102'}]}

## Get Job Result

In [13]:
get_result_url = f"{location}/results"
get_result_response = requests.request("GET", get_result_url)
get_result_response_json=json.loads(get_result_response.text)
print(json.dumps(get_result_response_json,indent=2))

{
  "stac": {
    "href": "http://localhost/temp//ZOO_DATA_dnbr_stac_21e9e69a-1268-11ed-a367-0242ac192202_0.json"
  }
}


## Inspect Job Result

In [14]:
result_url = get_result_response_json["stac"]["href"]
print(result_url)
result_response = requests.request("GET", result_url)
print(result_response.text)

http://localhost/temp//ZOO_DATA_dnbr_stac_21e9e69a-1268-11ed-a367-0242ac192202_0.json
Success!


## Undeploy the service

In [15]:
url = "http://localhost/eoepca/ogc-api/processes/dnbr"
response = requests.request("DELETE", url)
print(response.headers)
print(response.status_code)

{'Date': 'Tue, 02 Aug 2022 13:36:41 GMT', 'Server': 'Apache/2.4.41 (Ubuntu)', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive'}
204


In [16]:
for path, subdirs, files in os.walk(my_namespace_path):
    for name in files:
        print(os.path.join(path, name))

/opt/zooservices_namespaces/eoepca/UndeployProcess.py
/opt/zooservices_namespaces/eoepca/DeployProcess.py
/opt/zooservices_namespaces/eoepca/UndeployProcess.zcfg
/opt/zooservices_namespaces/eoepca/DeployProcess.zcfg
/opt/zooservices_namespaces/eoepca/__pycache__/UndeployProcess.cpython-38.pyc
/opt/zooservices_namespaces/eoepca/__pycache__/DeployProcess.cpython-38.pyc
